## Part 1: Preprocessing

In [53]:
#pip install tensorflow

In [54]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [55]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [56]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [57]:
# Create a list of at least 10 column names to use as X data
column_names = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[column_names]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [58]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [59]:
# Convert your X data to numeric data types however you see fit
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})
# Add new code cells as necessary
X_train['OverTime'].value_counts()

OverTime
0    780
1    322
Name: count, dtype: int64

In [60]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [61]:
from sklearn.preprocessing import OneHotEncoder

In [62]:
# Create a OneHotEncoder for the Department column
onehot_encoder = OneHotEncoder(handle_unknown='ignore' , sparse_output=False)

# Fit the encoder to the training data
onehot_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = onehot_encoder.transform(y_train[['Department']])
y_test_department = onehot_encoder.transform(y_test[['Department']])



In [63]:
# Create a OneHotEncoder for the Attrition column

onehot_encoder = OneHotEncoder(handle_unknown='ignore' , sparse_output=False)

# Fit the encoder to the training data
onehot_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = onehot_encoder.transform(y_train[['Attrition']])
y_test_attrition = onehot_encoder.transform(y_test[['Attrition']])

print(y_train_attrition)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Create, Compile, and Train the Model

In [64]:
# Find the number of columns in the X training data
num_columns = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer = layers.Dense(32, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer)

In [73]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu', 
                                       name='department_hidden')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department.shape[1], 
                                       activation='softmax', 
                                       name='department_output')(department_hidden_layer)


In [74]:
# Create a branch for Attrition
# with a hidden layer and an output layer

#Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu', 
                                      name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition.shape[1], 
                                      activation='sigmoid', 
                                      name='attrition_output')(attrition_hidden_layer)

In [75]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam', 
              loss={'department_output': 'categorical_crossentropy', 
                    'attrition_output': 'binary_crossentropy'}, 
              metrics={'department_output': 'accuracy', 
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 32)        │        352 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 32)        │      1,056 │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      1,056 │ dense_21[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      1,056 │ dense_21[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,685 (14.39 KB)

 Trainable params: 3,685 (14.39 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
# Train the model
history = model.fit(X_train_scaled, 
                    {'department_output': y_train_department, 
                     'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=10,
                    verbose=1)


Epoch 1/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9479 - department_output_accuracy: 0.8895 - loss: 0.4037
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9581 - department_output_accuracy: 0.8983 - loss: 0.3808
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9544 - department_output_accuracy: 0.9083 - loss: 0.3368
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9464 - department_output_accuracy: 0.8976 - loss: 0.3887
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9647 - department_output_accuracy: 0.9024 - loss: 0.3486
Epoch 6/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9492 - department_output_accuracy: 0.8952 - loss: 0.3689
Epoch 7/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9555 - department_output_accuracy: 0.9103 - loss: 0.3546
Epoch 8/100
1

In [78]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, 
                              {'department_output': y_test_department,  
                               'attrition_output': y_test_attrition}, 
                              verbose=1)
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7954 - department_output_accuracy: 0.4714 - loss: 5.6178  


[5.083349227905273, 0.804347813129425, 0.51902174949646]

In [81]:
# Print the accuracy for both department and attrition
print(f"Department prediction accuracy: {test_results[1]}")
print(f"Attrtion prediction accuracy: {test_results[2]}")

Department prediction accuracy: 0.804347813129425
Attrtion prediction accuracy: 0.51902174949646


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not always the best metric to use on this data because it is squed for the highest events that take place.  
2. I have used sigmoid for the yes or no output; softmax was used to assign a probability for department assessment.  
3. improve and adjust neuron layers to find a better combination; experiment with some additional models or combinations thereof; cleaning and add the data to better fit the models 